# Setup

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D, Dropout

# Preprocessing

In [ ]:
indices_train = np.load('../data/indices_train.npy')

X_full = np.stack(list(map(lambda index: np.load(f'../data/images_train/image_{index:03d}.npy'), range(40))))
y = np.stack(list(map(lambda index: np.load(f'../data/masks_train/mask_{index:03d}.npy'), range(40))))
# The labels do not change
y_train = y[indices_train[:, 0], 0, indices_train[:, 2], indices_train[:, 3]]

## Create 5x5 Patches

In [ ]:
KERNEL_RADIUS = 2
KERNEL_SIZE = KERNEL_RADIUS*2 + 1

expanded_indices_train = np.repeat(indices_train, (KERNEL_SIZE*KERNEL_SIZE), axis=0)

offsets = np.array([(i, j) for i in range(-KERNEL_RADIUS, KERNEL_RADIUS+1) for j in range(-KERNEL_RADIUS, KERNEL_RADIUS+1)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

X_train_5x5 = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
X_train_5x5 = np.reshape(X_train_5x5, ((X_train_5x5.shape[0] // (KERNEL_SIZE*KERNEL_SIZE)), KERNEL_SIZE, KERNEL_SIZE, -1))

# Calculate the mean and standard deviation of the data
mean = np.mean(X_train_5x5)
std = np.std(X_train_5x5)
# Normalize the data
X_train_5x5 = (X_train_5x5 - mean) / std

print(X_train_5x5.shape)

## Create 7x7 Patches

In [ ]:
KERNEL_RADIUS = 3
KERNEL_SIZE = KERNEL_RADIUS*2 + 1

expanded_indices_train = np.repeat(indices_train, (KERNEL_SIZE*KERNEL_SIZE), axis=0)

offsets = np.array([(i, j) for i in range(-KERNEL_RADIUS, KERNEL_RADIUS+1) for j in range(-KERNEL_RADIUS, KERNEL_RADIUS+1)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

X_train_7x7 = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
X_train_7x7 = np.reshape(X_train_7x7, ((X_train_7x7.shape[0] // (KERNEL_SIZE*KERNEL_SIZE)), KERNEL_SIZE, KERNEL_SIZE, -1))

# Calculate the mean and standard deviation of the data
mean = np.mean(X_train_7x7)
std = np.std(X_train_7x7)
# Normalize the data
X_train_7x7 = (X_train_7x7 - mean) / std

print(X_train_7x7.shape)

## Create 9x9 Patches

In [ ]:
KERNEL_RADIUS = 4
KERNEL_SIZE = KERNEL_RADIUS*2 + 1

expanded_indices_train = np.repeat(indices_train, (KERNEL_SIZE*KERNEL_SIZE), axis=0)

offsets = np.array([(i, j) for i in range(-KERNEL_RADIUS, KERNEL_RADIUS+1) for j in range(-KERNEL_RADIUS, KERNEL_RADIUS+1)])

# Compute new width and height indices with offsets and ensure they're within valid bounds
expanded_indices_train[:, 2] = np.clip(
    np.repeat(indices_train[:, 2], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 0], len(indices_train)), 0, 1023
)
expanded_indices_train[:, 3] = np.clip(
    np.repeat(indices_train[:, 3], (KERNEL_SIZE*KERNEL_SIZE)) + np.tile(offsets[:, 1], len(indices_train)), 0, 1023
)

X_train_9x9 = X_full[expanded_indices_train[:, 0], :, expanded_indices_train[:, 2], expanded_indices_train[:, 3]]
X_train_9x9 = np.reshape(X_train_9x9, ((X_train_9x9.shape[0] // (KERNEL_SIZE*KERNEL_SIZE)), KERNEL_SIZE, KERNEL_SIZE, -1))

# Calculate the mean and standard deviation of the data
mean = np.mean(X_train_9x9)
std = np.std(X_train_9x9)
# Normalize the data
X_train_9x9 = (X_train_9x9 - mean) / std

print(X_train_9x9.shape)

# Model Training

In [ ]:
def get_run_logdir(model_id):
    return os.path.join(os.curdir, f'cnn_logs/model_{model_id}')

earlystopping_cb = keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./cnn_logs --port=6006

In [ ]:
model = {}

## Model 1: 5x5 Input, 3x3x3 Kernel, 10 Dense

In [ ]:
i = 1
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 2: 5x5 Input, 3x3x8 Kernel, 10 Dense

In [ ]:
i = 2
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 3: 5x5 Input, 3x3x12 Kernel, 10 Dense

In [ ]:
i = 3
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 4: 5x5 Input, 3x3x8 Kernel, 50 Dense, 10 Dense

In [ ]:
i = 4
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 5: 5x5 Input, 3x3x8 Kernel, 0.2 Dropout, 50 Dense, 10 Dense

In [ ]:
i = 5
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 6: 5x5 Input, 3x3x8 Kernel, 0.1 Dropout, 50 Dense, 10 Dense

In [ ]:
i = 6
model[i] = Sequential([
    InputLayer(input_shape=[5, 5, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_5x5, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 7: 7x7 Input, 3x3x3 Kernel, 10 Dense

In [ ]:
i = 7
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 8: 7x7 Input, 3x3x8 Kernel, 10 Dense

In [ ]:
i = 8
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 9: 7x7 Input, 3x3x8 Kernel, 50 Dense, 10 Dense

In [ ]:
i = 9
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 10: 7x7 Input, 3x3x8 Kernel, 0.2 Dropout, 50 Dense, 10 Dense

In [ ]:
i = 10
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 11: 7x7 Input, 3x3x8 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 11
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 12: 7x7 Input, 3x3x12 Kernel, 0.1 Dropout, 100 Dense, 0.1 Dropout, 50 Dense, 10 Dense

In [ ]:
i = 12
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 13: 7x7 Input, 3x3x12 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 50 Dense, 10 Dense

In [ ]:
i = 13
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 14: 7x7 Input, 3x3x12 Kernel, 20 Dense

In [ ]:
i = 14
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(20, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 15: 7x7 Input, 3x3x12 Kernel, 0.05 Dropout, 20 Dense

In [ ]:
i = 15
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.05),
    Dense(20, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 16: 7x7 Input, 3x3x12 Kernel, 0.05 Dropout, 40 Dense

In [ ]:
i = 16
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.05),
    Dense(40, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 17: 7x7 Input, 3x3x12 Kernel, 0.1 Dropout, 80 Dense

In [ ]:
i = 17
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.05),
    Dense(80, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 18: 7x7 Input, 3x3x12 Kernel, 0.15 Dropout, 120 Dense

In [ ]:
i = 18
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.15),
    Dense(120, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 19: 9x9 Input, 3x3x3 Kernel, 10 Dense

In [ ]:
i = 19
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=3, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 20: 9x9 Input, 3x3x8 Kernel, 10 Dense

In [ ]:
i = 20
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 21: 9x9 Input, 5x5x8 Kernel, 10 Dense

In [ ]:
i = 21
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=5, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 22: 9x9 Input, 5x5x8 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 10 Dense

In [ ]:
i = 22
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=5, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 23: 9x9 Input, 3x3x8 Kernel, 0.1 Dropout, 100 Dense, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 23
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 24: 9x9 Input, 3x3x8 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 50 Dense, 0.2 Dropout, 10 Dense

In [ ]:
i = 24
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 25: 9x9 Input, 3x3x8 Kernel, 0.3 Dropout, 100 Dense, 0.3 Dropout, 50 Dense, 0.3 Dropout, 10 Dense

In [ ]:
i = 25
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 26: 9x9 Input, 3x3x8 Kernel, 0.1 Dropout, 30 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 26
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(30, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 27: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 10 Dense

In [ ]:
i = 27
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 28: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 20 Dense, 10 Dense

In [ ]:
i = 28
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(20, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 29: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 29
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 30: 9x9 Input, 3x3x8 Kernel, 3x3x2 Kernel, 0.1 Dropout, 100 Dense, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 30
model[i] = Sequential([
    InputLayer(input_shape=[9, 9, 10]),
    Conv2D(filters=8, kernel_size=3, strides=1, activation='relu'),
    Conv2D(filters=2, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_9x9, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 31: 7x7 Input, 3x3x12 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 31
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=12, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 32: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 50 Dense, 0.1 Dropout, 10 Dense

In [ ]:
i = 32
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(50, activation='relu'),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 33: 7x7 Input, 3x3x20 Kernel, 0.2 Dropout, 100 Dense, 0.2 Dropout, 50 Dense, 0.2 Dropout, 10 Dense

In [ ]:
i = 33
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 34: 7x7 Input, 3x3x20 Kernel, 0.3 Dropout, 100 Dense, 0.3 Dropout, 50 Dense, 0.3 Dropout, 10 Dense

In [ ]:
i = 34
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(50, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 35: 7x7 Input, 3x3x20 Kernel, 0.25 Dropout, 100 Dense, 10 Dense

In [ ]:
i = 35
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.25),
    Dense(100, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 36: 7x7 Input, 3x3x20 Kernel, 0.2 Dropout, 75 Dense, 25 Dense

In [ ]:
i = 36
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.2),
    Dense(75, activation='relu'),
    Dense(25, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 37: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 20 Dense, 10 Dense

In [ ]:
i = 37
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(20, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 38: 7x7 Input, 3x3x20 Kernel, 10 Dense

In [ ]:
i = 38
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 39: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 10 Dense

In [ ]:
i = 39
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 40: 7x7 Input, 3x3x20 Kernel, 0.15 Dropout, 20 Dense

In [ ]:
i = 40
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.15),
    Dense(20, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

## Model 41: 7x7 Input, 3x3x20 Kernel, 0.1 Dropout, 25 Dense

In [ ]:
i = 41
model[i] = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(25, activation='relu'),
    Dense(1, activation='relu')
])
model[i].summary()
model[i].compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model[i].fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(i)), earlystopping_cb])
model[i].save(f'cnn_models/model_{i}')

# Best Model: 39

Very good fit, comparably few parameters, fast training.

In [ ]:
model = Sequential([
    InputLayer(input_shape=[7, 7, 10]),
    Conv2D(filters=20, kernel_size=3, strides=1, activation='relu'),
    Flatten(),
    Dropout(0.1),
    Dense(10, activation='relu'),
    Dense(1, activation='relu')
])
model.summary()
model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["mean_absolute_error"])
history = model.fit(
    X_train_7x7, 
    y_train, 
    epochs=1000, 
    batch_size=256, 
    validation_split=0.2,
    verbose=0,
    callbacks=[keras.callbacks.TensorBoard(get_run_logdir(0)), earlystopping_cb])
model.save('../models/cnn')